$$\textbf{Chad Schupbach}$$

***

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from src import utils

Using TensorFlow backend.


A common CNN feature learning architecture is similar to that shown above, where feature maps are constructed with ReLU-activated convolutional layers and intermittent max pooling. It's also fairly common to normalize the batch of input layers prior to convolution in the feature learning segment.

## Classification

In the classification segment of the network, we flatten the final laywe shift our focus to classification using fully connected (FC) layers.

### Fully Connected (FC) Layers

In [2]:
x_train, y_train, x_test, y_test, input_shape = utils.load_mnist(method='keras')

In [3]:
# model.summary()

$$\begin{aligned} \underset{28 \times 28 \times 1}{ \ \ \textsf{input}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{28 \times 28 \times 16}{ \ \ \textsf{conv1}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{28 \times 28 \times 16}{\textsf{conv2}^\dagger} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto}\\[5pt]
\underset{14 \times 14 \times 16}{ \, \ \textsf{pool1}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{14 \times 14 \times 32}{\textsf{conv3}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{14 \times 14 \times 32}{\textsf{conv4}^\dagger} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto}\\[5pt]
\underset{7 \times 7 \times 32}{ \, \ \textsf{pool2}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{7 \times 7 \times 64}{ \, \ \textsf{conv5}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{7 \times 7 \times 64}{ \, \ \textsf{conv6}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto}\\[5pt] \underset{7 \times 7 \times 64}{ \, \ \textsf{conv7}^\dagger} \ \ _{ + \ \ \text{flatten}_{1600} \ + \ \text{dropout}_{0.25} \ + \ \text{dense}_{128} \ + \ \text{ReLU} \ \ \longmapsto}\\[10pt]
\underset{1 \times 1 \times 128}{\textsf{FC6}} \ \ _{ + \ \ \text{dropout}_{0.5} \ + \ \text{dense}_{128} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 128}{\textsf{FC7}} \ \ _{ + \ \ \text{dropout}_{0.5} \ + \ \text{dense}_{10} \ + \ \text{Softmax} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 10}{\textsf{output}} \end{aligned}$$

Here, we introduce FC layers by building a network beginning with feature learning. We initialize a Keras sequential model object as follows:

Beginning with the architecture

$$\underset{30 \times 30 \times 1}{ \ \ \textsf{input}^+} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{30 \times 30 \times 32}{ \ \ \textsf{conv1}^+} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{28 \times 28 \times 64}{\textsf{conv2}} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto} \ \ \underset{16 \times 16 \times 64}{ \, \ \textsf{pool1}^\ddagger}$$

we add layers as follows:

In [3]:
model = Sequential()

In [4]:
model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

$$\underset{16 \times 16 \times 64}{ \, \ \textsf{pool1}^\ddagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{14 \times 14 \times 64}{\textsf{conv3}} \ \ _{ \ast \ \ \text{max pool}_{2 \times 2} \ \ \longmapsto} \ \ \underset{8 \times 8 \times 64}{ \, \ \textsf{pool2}^\ddagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{7 \times 7 \times 64}{ \, \ \textsf{conv4}^\dagger}$$

In [5]:
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
# model.add(BatchNormalization())

Here we switch over to classification by vectorizing `conv5` as follows:

$$\underset{7 \times 7 \times 64}{ \, \ \textsf{conv4}^\dagger} \ \ _{\circledast \ \ \text{conv}_{3 \times 3} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{5 \times 5 \times 64}{ \, \ \textsf{conv5}^\dagger} \ \ _{ + \ \ \text{flatten}_{1600} \ + \ \text{dropout}_{0.25} \ + \ \text{dense}_{156} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 156}{\textsf{FC6}}$$

In [6]:
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(ZeroPadding2D(padding=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))

$$\underset{1 \times 1 \times 156}{\textsf{FC6}} \ \ _{ + \ \text{dropout}_{0.5} \ + \ \text{dense}_{156} \ + \ \text{ReLU} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 156}{\textsf{FC7}} \ \ _{ + \ \text{dropout}_{0.5} \ + \ \text{dense}_{10} \ + \ \text{Softmax} \ \ \longmapsto} \ \ \underset{1 \times 1 \times 10}{\textsf{output}}$$

In [7]:
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)          1

In [ ]:
# fp = 'models/best_weights.hdf5'
# checkpoint = ModelCheckpoint(fp, monitor='val_accuracy', save_best_only=True,
#                              save_weights_only=True, mode='max')

In [8]:
model.compile(optimizer='nadam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, batch_size=128, epochs=2, verbose=1,
          validation_split=0.2, initial_epoch=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.4966 - accuracy: 0.8381 - val_loss: 0.0673 - val_accuracy: 0.9805
Epoch 2/2
48000/48000 [==============================] - 52s 1ms/sample - loss: 0.1167 - accuracy: 0.9694 - val_loss: 0.0479 - val_accuracy: 0.9866


In [10]:
import numpy as np
import matplotlib.pyplot as plt

arr = np.array([
    [0.4583, 0.8572, 0.0730, 0.9807],
    [0.1046, 0.9735, 0.0512, 0.9858],
    [0.0736, 0.9824, 0.0437, 0.9881],
    [0.0604, 0.9842, 0.0519, 0.9869],
    [0.0525, 0.9870, 0.0328, 0.9908],
    [0.0448, 0.9886, 0.0461, 0.9906],
    [0.0451, 0.9883, 0.0405, 0.9900],
    [0.0397, 0.9902, 0.0375, 0.9908],
    [0.0398, 0.9899, 0.0419, 0.9915],
    [0.0385, 0.9900, 0.0642, 0.9860],
    [0.0395, 0.9901, 0.0477, 0.9908],
    [0.0298, 0.9924, 0.0591, 0.9895],
    [0.0341, 0.9912, 0.0415, 0.9914],
    [0.0316, 0.9919, 0.0417, 0.9908],
    [0.0293, 0.9928, 0.0371, 0.9915],
    [0.0310, 0.9923, 0.0379, 0.9910],
    [0.0304, 0.9923, 0.0412, 0.9925],
    [0.0288, 0.9928, 0.0576, 0.9890],
    [0.0282, 0.9934, 0.0421, 0.9925],
    [0.0286, 0.9924, 0.0522, 0.9902],
    [0.0301, 0.9930, 0.0511, 0.9898],
    [0.0266, 0.9932, 0.0459, 0.9897],
    [0.0243, 0.9940, 0.0494, 0.9925],
    [0.0275, 0.9934, 0.0537, 0.9903],
    [0.0261, 0.9934, 0.0435, 0.9930],
    [0.0264, 0.9938, 0.0483, 0.9909],
    [0.0216, 0.9947, 0.0526, 0.9912],
    [0.0260, 0.9938, 0.0441, 0.9918],
    [0.0267, 0.9937, 0.0476, 0.9918],
    [0.0244, 0.9942, 0.0559, 0.9904],
    [0.0281, 0.9936, 0.0619, 0.9909],
    [0.0269, 0.9935, 0.0666, 0.9913],
    [0.0347, 0.9920, 0.0548, 0.9881],
    [0.0318, 0.9928, 0.0556, 0.9906],
    [0.0248, 0.9943, 0.0445, 0.9912],
    [0.0261, 0.9936, 0.0579, 0.9924],
    [0.0221, 0.9950, 0.0581, 0.9914],
    [0.0188, 0.9957, 0.0693, 0.9912],
    [0.0216, 0.9948, 0.0700, 0.9915],
    [0.0309, 0.9932, 0.0572, 0.9906],
    [0.0234, 0.9942, 0.0718, 0.9917],
    [0.0320, 0.9930, 0.0601, 0.9908],
    [0.0235, 0.9952, 0.0551, 0.9920],
    [0.0237, 0.9946, 0.0818, 0.9904],
    [0.0361, 0.9921, 0.0654, 0.9923],
    [0.0191, 0.9953, 0.0763, 0.9923],
    [0.0296, 0.9942, 0.0631, 0.9925],
    [0.0198, 0.9956, 0.0727, 0.9925],
    [0.0307, 0.9933, 0.0737, 0.9914],
    [0.0218, 0.9949, 0.0772, 0.9921]
])

In [ ]:
plt.plot(arr[1:,[1,3]])

In [11]:
# model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1,
#           validation_split=0.2, initial_epoch=10)

Train on 48000 samples, validate on 12000 samples
Epoch 11/50
48000/48000 [==============================] - 49s 1ms/step - loss: 0.0395 - accuracy: 0.9901 - val_loss: 0.0477 - val_accuracy: 0.9908
Epoch 12/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0298 - accuracy: 0.9924 - val_loss: 0.0591 - val_accuracy: 0.9895
Epoch 13/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0341 - accuracy: 0.9912 - val_loss: 0.0415 - val_accuracy: 0.9914
Epoch 14/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0316 - accuracy: 0.9919 - val_loss: 0.0417 - val_accuracy: 0.9908
Epoch 15/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0293 - accuracy: 0.9928 - val_loss: 0.0371 - val_accuracy: 0.9915
Epoch 16/50
48000/48000 [==============================] - 51s 1ms/step - loss: 0.0310 - accuracy: 0.9923 - val_loss: 0.0379 - val_accuracy: 0.9910
Epoch 17/50
48000/48000 [==============================] - 51s

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=10)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=30, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=20)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=40, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=30)

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=50, verbose=1, callbacks=[checkpoint],
          validation_data=(x_test, y_test), initial_epoch=40)

In [ ]:
model.load_weights(fp)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])